In [1]:
import pandas as pd
import numpy as np
import regex as re
import warnings
warnings.filterwarnings('ignore') 
import pylab as plt
import seaborn as sns


In [2]:
sharkis= pd.read_csv('../data/attacks.csv', encoding='latin1')

shark=sharkis.copy()

shark.head()


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


## Exploracion del archivo y estudio

In [3]:
shark.shape #dimensiones

(25723, 24)

In [4]:
shark.columns # indices

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
shark.info(memory_usage='deep') #tipo de dato que tenemos entre manos 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [6]:
# vamos a corregir los nombres de las columnas:

shark.rename(columns={'Sex ':'Sex', 'Species ':'Species','Fatal(Y/N)':'Fatality'})
shark.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [7]:
# No se porque Fatal(Y/N) no me lo corrige asi que a lo bestia:

shark.columns.values[12]='Fatality'

shark.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatality', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [8]:
# vamos a pasa 'Year' de float a int:
shark.Year.fillna(0,inplace=True)

year=shark.Year.astype(dtype='int32')

shark.Year=year

In [9]:
# Quitamos puntos de Case.number.n:

columnas=[]

for c in shark.columns:
    c= c.replace('.',' ')
    c=c.replace(' ','')
    columnas.append(c)

shark.columns=columnas

In [10]:
# Veamos como ha quedado:

shark.head()

,CaseNumber,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,InvestigatororSource,pdf,hrefformula,href,CaseNumber1,CaseNumber2,originalorder,Unnamed:22,Unnamed:23
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


## Limpieza:

### Valores nulos:

In [11]:
nulos =shark.isna().mean() * 100  # porcentaje de nulos por columna 

columna_nula= nulos[nulos>0].sort_values(ascending=False)
columna_nula

Unnamed:22              99.996112
Unnamed:23              99.992225
Time                    88.539439
Species                 86.533453
Age                     86.506240
Sex                     77.697003
Activity                77.615364
Location                77.599813
Fatality                77.595926
Area                    77.269370
Name                    76.316915
Country                 75.694903
Injury                  75.609377
InvestigatororSource    75.566614
Type                    75.516075
hrefformula             75.504412
Date                    75.500525
pdf                     75.500525
href                    75.500525
CaseNumber1             75.500525
CaseNumber2             75.500525
originalorder           75.473312
CaseNumber              66.170353
dtype: float64

In [12]:
shark.dropna(thresh=len(shark.columns) / 3, inplace=True) 

In [13]:
shark.loc[:, ['Unnamed: 22', 'Unnamed: 23']] = 0 #igualo a 0 porque no tienen contenido representativo

### Valores duplicados

In [14]:
shark.duplicated().any()

False

In [15]:
# No hay valores duplicados

### Valores constantes

In [16]:
#miramos los contantes que no tengan numericos:

varianza= []

for c in shark.select_dtypes(include=np.number):

    if len (shark[c].unique()) == 1:
        varianza.append(c)

varianza

['Unnamed: 22', 'Unnamed: 23']

No hay valores constantes

In [17]:
data=shark.copy()


### Outliers:

No tiene sentido, los datos son muy dispare y como hemos visto no hay varianza

## Estudio de columnas por intereses:

Vamos a volver a chequear el df para ver como ha quedado tras una limpieza preliminar. Seguro que habra columnas que necesitaran limpiar nombre o fechas.

In [18]:
data.head()

,CaseNumber,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,pdf,hrefformula,href,CaseNumber1,CaseNumber2,originalorder,Unnamed:22,Unnamed:23,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN,0,0
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN,0,0
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN,0,0
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN,0,0
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN,0,0


In [19]:
# Me gustaría centrarme en las columnas de Sex, Fatality,Activity ya que son las que necesito para mi objetivo. 
# Si me sale todo bien seguire con Country,Area,Localotation para ampliar el analisis 🤞🤞.

In [20]:
# Uso .value_count() para ver que valores tiene dentro. Así puedo apreciar mejor que tengo para limpiar

data.Sex.value_counts() 

Sex
M      5094
F       637
M         2
N         2
lli       1
.         1
Name: count, dtype: int64

In [21]:
#Vamos a corregirlo
data.Sex=np.where((data.Sex == 'M')|(data.Sex=='N')|(data.Sex=='lli'), 'M',np.where(data.Sex== 'F','F','Unkown'))
data.Sex.value_counts()

Sex
M         5097
F          637
Unkown     568
Name: count, dtype: int64

In [22]:
#seguimos con Fatality:

data.Fatality.value_counts()

Fatality
N          4293
Y          1388
UNKNOWN      71
 N            7
M             1
2017          1
N             1
y             1
Name: count, dtype: int64

In [23]:
data.Fatality.unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y'],
      dtype=object)

In [24]:
data.Fatality = data['Fatality'].str.upper()
data.Fatality = np.where((data.Fatality == ' N') | (data.Fatality == 'M') | (data.Fatality == 'N'), 'N', 
                        np.where(data.Fatality == 'Y', 'Y', 'Unknown'))

data.Fatality.value_counts()

Fatality
N          4301
Y          1389
Unknown     612
Name: count, dtype: int64

In [25]:
# Ahora activity:
data.Activity.value_counts()

Activity
Surfing                                   971
Swimming                                  869
Fishing                                   431
Spearfishing                              333
Bathing                                   162
                                         ... 
Playing with a frisbee in the shallows      1
Sinking of the ferryboat Dumaguete          1
Wreck of the Storm King                     1
Feeding mullet to sharks                    1
Wreck of  large double sailing canoe        1
Name: count, Length: 1532, dtype: int64

In [26]:
#voy a crear una funcion para intentar igualar todas las actividades(sin la ayuda de chat GPT no hubiese sido posible 🙏)

def activity(x):
    dict_activity= {"Pesca":re.search(".*[Ff](ishing|ISHING).*",str(x)),
                    "Actividades con tabla(surf,kite,etc...)":re.search(".*[sS](urfing|SURF)|[Oo](verb|VERB).*",str(x)),
                    "Natación":re.search(".*[Ss](wimming|WIMMing).*",str(x)),
                    "Kite":re.search(".*[Kk](ite|ITE).*",str(x)),
                    "Actividades en aguas poco profundas":re.search(".*[Ww](alking|ALKING)|[Ss](tand|TAND).*",str(x)),
                    "Diferentes actividades en barco":re.search(".*[Bb](oat|OAT).*",str(x)),
                    "Actividades con tiburones":re.search(".*[Ss](hark|HARK).*",str(x)),
                    "Buceo":re.search(".*[Dd](iving|IVING).*",str(x)),
                    "Remo":re.search(".*[Pp](addl|ADDL).*",str(x)),
                    "Baño":re.search(".*[Bb](athing|ATHING)|[Ff](loat|LOAT).*",str(x)),
                    "Salto al agua":re.search(".*[Jj](ump|UMP).*",str(x)),
                    "Naufragios":re.search(".*[Ww](reck|RECK ).*",str(x))}
    for key,values in dict_activity.items():

        if values:
            return key
    return "Otras"

data.Activity=data.Activity.apply(activity)

In [27]:
data.Activity.value_counts()

Activity
Otras                                      1563
Actividades con tabla(surf,kite,etc...)    1224
Pesca                                      1172
Natación                                   1092
Buceo                                       510
Baño                                        231
Actividades en aguas poco profundas         165
Actividades con tiburones                   145
Diferentes actividades en barco             102
Naufragios                                   41
Remo                                         33
Salto al agua                                15
Kite                                          9
Name: count, dtype: int64

In [28]:
#un poco de correcciones en Country:

data.Country.value_counts().tail()

Country
MALDIVE ISLANDS           1
NICARAGUA                 1
NORTH SEA                 1
RED SEA / INDIAN OCEAN    1
CEYLON (SRI LANKA)        1
Name: count, dtype: int64

In [29]:
data.Country=data.Country.str.upper()
data.Country=data.Country.str.strip()
data.Country=data.Country.str.replace(r'\b(NORTH|SOUTH|EAST|WEST)\b','',regex=True)
data.Country=data.Country.replace('?','Aprox')


data.Country.value_counts().head(50)

Country
USA                 2229
AUSTRALIA           1338
 AFRICA              579
PAPUA NEW GUINEA     134
NEW ZEALAND          128
BRAZIL               112
BAHAMAS              109
MEXICO                90
ITALY                 71
FIJI                  65
PHILIPPINES           62
REUNION               60
NEW CALEDONIA         53
CUBA                  46
MOZAMBIQUE            45
SPAIN                 44
EGYPT                 40
INDIA                 40
JAPAN                 34
CROATIA               34
PANAMA                32
SOLOMON ISLANDS       30
IRAN                  29
JAMAICA               27
GREECE                25
FRENCH POLYNESIA      25
HONG KONG             24
INDONESIA             23
ENGLAND               23
PACIFIC OCEAN         19
TONGA                 18
ATLANTIC OCEAN        17
 ATLANTIC OCEAN       17
COSTA RICA            17
BERMUDA               16
VIETNAM               15
SRI LANKA             14
VANUATU               14
MARSHALL ISLANDS      13
FRANCE           

In [30]:
#Toca Type, aunque es lo mismo que Activity

data.Type.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [31]:
#vamos a corregir:

shark.Type=shark.Type.str.strip()
shark.Type=shark.Type.str.replace('Boating','Boat')
shark.Type=shark.Type.str.replace('Invalid','Unknown')
shark.Type=shark.Type.str.replace ('Boatomg','Boat')
shark.Type=shark.Type.str.replace('Questionable','Unknown')
shark.Type.fillna('Unknown',inplace=True)
shark.Type.value_counts()

Type
Unprovoked      4595
Provoked         574
Unknown          553
Boat             341
Sea Disaster     239
Name: count, dtype: int64

In [32]:
data.Year.value_counts()

Year
2015    143
2017    136
2016    130
2011    128
0       127
       ... 
1787      1
1786      1
1785      1
1784      1
5         1
Name: count, Length: 249, dtype: int64

In [33]:
# Quitamos los valores raros(he usado antes .unique() y he visto fechas malas pero es muy largo el array y no lo vot a mostrar):

arr=data.Year
patron=r'\b\d{4}\b'


date = [int(año) if re.match(patron, str(año)) else 0000 for año in arr]

data.Year=date